# Лабораторная работа 1
Выполнил студент группы 0385 Иванов Серафим

In [3]:
import xml.etree.ElementTree as ET
from collections import defaultdict
import urllib.request
from tqdm import tqdm
import bz2
import os
import pickle
import re
import shutil
import zipfile

## Скачивание нужных файлов

In [4]:
import os

# Папка для сохранения файлов
output_dir = 'downloads'
os.makedirs(output_dir, exist_ok=True)

# Ссылки на файлы
urls = {
    'opencorpora_dict.xml.bz2': 'https://opencorpora.org/files/export/dict/dict.opcorpora.xml.bz2',
    'сrime_and_punishment.txt': 'https://bookex.info/uploads/public_files/2023-02/prestuplenie-i-nakazanie_-fedor-dostoevskij.txt',
    'war_and_peace.txt': 'https://gist.github.com/Semionn/bdcb66640cc070450817686f6c818897/raw/f9e8c888a771dd96f54562a9b050acd1138cc7a9/war_and_peace.ru.txt'
}

# Скачивание файлов
for filename, url in urls.items():
    output_path = os.path.join(output_dir, filename)
    urllib.request.urlretrieve(url, output_path)
    print(f'Файл "{filename}" успешно скачан в "{output_path}"')


Файл "opencorpora_dict.xml.bz2" успешно скачан в "downloads\opencorpora_dict.xml.bz2"
Файл "сrime_and_punishment.txt" успешно скачан в "downloads\сrime_and_punishment.txt"
Файл "war_and_peace.txt" успешно скачан в "downloads\war_and_peace.txt"


In [5]:
# Перемещаем все файлы из downloads в родительскую папку
parent_dir = os.path.dirname(output_dir)

for filename in os.listdir(output_dir):
    src_path = os.path.join(output_dir, filename)
    dst_path = os.path.join(parent_dir, filename)
    if os.path.isfile(src_path):
        shutil.move(src_path, dst_path)
        print(f'file "{filename}" mod=vet into "{dst_path}"')

file "opencorpora_dict.xml.bz2" mod=vet into "opencorpora_dict.xml.bz2"
file "war_and_peace.txt" mod=vet into "war_and_peace.txt"
file "сrime_and_punishment.txt" mod=vet into "сrime_and_punishment.txt"


## Обработка словаря

In [6]:
BZ2_PATH = "dict.opcorpora.xml.bz2"   # сжатый словарь
XML_PATH = "dict.opcorpora.xml"       # распакованный xml
PICKLE_PATH = "lemma_dict.pkl"        # куда сохраняем готовый словарь

if not os.path.exists(XML_PATH) and os.path.exists(BZ2_PATH):
    print("Unpacking dict...")
    with bz2.open(BZ2_PATH, "rb") as f_in, open(XML_PATH, "wb") as f_out:
        f_out.write(f_in.read())
    print("Done.")

In [ ]:
GRAMM_TO_SIMPLE = {
    "NOUN": "S",
    "S": "S",
    "NPRO": "NI",

    "ADJF": "A",
    "ADJS": "A",
    "COMP": "A",

    "VERB": "V",
    "INFN": "V",
    "V": "V",

    "PRTF": "PRTF",  # причастие полное
    "PRTS": "PRTS",  # причастие краткое
    "GRND": "GRND",  # деепричастие

    "ADVB": "ADV",
    "ADV": "ADV",

    "NUMR": "NUM",
    "NUM": "NUM",

    "PREP": "PR",
    "PR": "PR",
    "CONJ": "CONJ",
    "PRCL": "PART",
    "INTJ": "INTJ",
}

POS_GRAMMEMES = set(GRAMM_TO_SIMPLE.keys())

In [8]:
def normalize(word: str) -> str:
    """Lower case + ё->е"""
    return word.lower().replace("ё", "е")

In [9]:
print("Counting <lemma>...")
lemma_count = 0
for _, elem in ET.iterparse(XML_PATH, events=("end",)):
    if elem.tag == "lemma":
        lemma_count += 1
    elem.clear()
print(f"Total: {lemma_count:,}")

Counting <lemma>...
Total: 391,845


In [10]:
lemma_dict = {}
lemma_dict_raw = {}        # lemma_id -> (lemma_text, pos_simple, forms)
form2lemma = {}            # form_norm -> [lemma_id1, lemma_id2, ...]
links = {}                 # to_id -> from_id (связи из XML)

VALID_LINK_TYPES = {"3", "5", "6"}  # инфинитив, деепричастие, причастие

print("Parsing XML and collecting lemmas...")
it = ET.iterparse(XML_PATH, events=("end",))
with tqdm(total=lemma_count, unit="lemma") as pbar:
    for event, elem in it:
        if elem.tag == "lemma":
            lemma_id = elem.get("id")
            l_elem = elem.find("l")
            if l_elem is not None:
                lemma_text = l_elem.get("t") or ""
                pos_simple = None
                for g in l_elem.findall("g"):
                    v = g.get("v")
                    if v in POS_GRAMMEMES:
                        pos_simple = GRAMM_TO_SIMPLE[v]
                        break
                if pos_simple is None:
                    pos_simple = "?"

                forms = [lemma_text] + [f.get("t") for f in elem.findall("f") if f.get("t")]
                lemma_dict_raw[lemma_id] = (lemma_text, pos_simple, forms)

                for form in forms:
                    norm = normalize(form)
                    form2lemma.setdefault(norm, []).append(lemma_id)

            elem.clear()
            pbar.update(1)

        elif elem.tag == "link":
            link_type = elem.get("type")
            if link_type in VALID_LINK_TYPES:
                from_id = elem.get("from")
                to_id = elem.get("to")
                links[to_id] = from_id
            elem.clear()

# --- функция для подъема по цепочке до базовой леммы ---
def get_base_id(lemma_id, links):
    seen = set()
    while lemma_id in links and lemma_id not in seen:
        seen.add(lemma_id)
        lemma_id = links[lemma_id]
    return lemma_id

print("Building final lemma_dict...")
with tqdm(total=len(form2lemma), unit="form") as pbar:
    for form_norm, lemma_ids in form2lemma.items():
        variants = set()  # используем set для уникальности
        for lemma_id in lemma_ids:
            lemma_text_base, pos_simple_orig, _ = lemma_dict_raw[lemma_id]
            # лемму берём от базовой формы по цепочке ссылок
            base_id = get_base_id(lemma_id, links)
            lemma_text_base = lemma_dict_raw[base_id][0]
            lemma_norm = normalize(lemma_text_base)
            variants.add((lemma_norm, pos_simple_orig))
        # сохраняем как список
        lemma_dict[form_norm] = sorted(variants)
        pbar.update(1)

print("The dict is ready, total keys:", len(lemma_dict))


Parsing XML and collecting lemmas...


100%|██████████| 391845/391845 [00:37<00:00, 10454.36lemma/s]


Building final lemma_dict...


100%|██████████| 3060604/3060604 [00:13<00:00, 221335.37form/s]

The dict is ready, total keys: 3060604


In [11]:
with open(PICKLE_PATH, "wb") as f:
    pickle.dump(lemma_dict, f, protocol=4)
print("Saved to", PICKLE_PATH)

Saved to lemma_dict.pkl


In [12]:
lemma_dict

{'еж': [('еж', 'S')],
 'ежа': [('еж', 'S'), ('ежить', 'GRND')],
 'ежу': [('еж', 'S'), ('ежить', 'V')],
 'ежом': [('еж', 'S'), ('ежом', 'ADV')],
 'еже': [('еж', 'S')],
 'ежи': [('еж', 'S'), ('ежи', 'S')],
 'ежей': [('еж', 'S')],
 'ежам': [('еж', 'S')],
 'ежами': [('еж', 'S')],
 'ежах': [('еж', 'S')],
 'ежик': [('ежик', 'S')],
 'ежика': [('ежик', 'S')],
 'ежику': [('ежик', 'S')],
 'ежиком': [('ежик', 'S'), ('ежиком', 'ADV')],
 'ежике': [('ежик', 'S')],
 'ежики': [('ежик', 'S')],
 'ежиков': [('ежик', 'S')],
 'ежикам': [('ежик', 'S')],
 'ежиками': [('ежик', 'S')],
 'ежиках': [('ежик', 'S')],
 'ежистый': [('ежистый', 'A')],
 'ежистого': [('ежистый', 'A')],
 'ежистому': [('ежистый', 'A')],
 'ежистым': [('ежистый', 'A')],
 'ежистом': [('ежистый', 'A')],
 'ежистая': [('ежистый', 'A')],
 'ежистой': [('ежистый', 'A')],
 'ежистую': [('ежистый', 'A')],
 'ежистою': [('ежистый', 'A')],
 'ежистое': [('ежистый', 'A')],
 'ежистые': [('ежистый', 'A')],
 'ежистых': [('ежистый', 'A')],
 'ежистыми': [('ежи

## Обработка текста

Загружаем готовый словарь

In [13]:
PICKLE_PATH = "lemma_dict.pkl"
with open(PICKLE_PATH, "rb") as f:
    lemma_dict = pickle.load(f)
print("Dict loaded, keys:", len(lemma_dict))

Dict loaded, keys: 3060604


Функции для лемматизации

In [ ]:
# глобальные счётчики
total_tokens = 0
not_found_tokens = 0

def lemmatize_token(token: str):
    """
    Returns the token in the format:
      token{lemma=POS}  -- если один вариант
      token{{lemma1=POS1|lemma2=POS2|...}}  -- если несколько вариантов
    Updates global counters.
    """
    global total_tokens, not_found_tokens
    total_tokens += 1

    norm = normalize(token)
    if norm in lemma_dict:
        variants = lemma_dict[norm]
        if len(variants) == 1:
            lemma, pos = variants[0]
            return f"{token}{{{lemma}={pos}}}"
        else:
            # несколько вариантов -> перечисляем через |
            options = "|".join(f"{lemma}={pos}" for lemma, pos in variants)
            return f"{token}{{{options}}}"
    else:
        not_found_tokens += 1
        return f"{token}{{{norm}=??}}"

def process_sentence(sentence: str):
    """
    1. Remove all punctuation
    2. Split the sentence into tokens
    3. Lemmatize each token (updates counters)
    4. Join tokens back into a string
    """
    clean = re.sub(r"[^\w\s]", "", sentence, flags=re.UNICODE)
    tokens = clean.split()
    return " ".join(lemmatize_token(tok) for tok in tokens)

Использование

In [39]:
# Сброс глобальных счётчиков перед обработкой
total_tokens = 0
not_found_tokens = 0

input_text = """Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.
Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.
"""

# input_text = """Не существует дедуктики, адекватной множеству истин арифметики"""

# input_text = """Пятеро путешественников, уставшие, но улыбающиеся, медленно поднимались по горной тропе, осторожно ступая на скользкие камни.
# Они шли, переговариваясь и смеясь, ведь впереди их ждала цель, о которой они мечтали.
# Эй, неужели дорога наконец закончилась?"""


print("Input:")
print(input_text)

print("\nOutput:")
for line in input_text.strip().split("\n"):
    print(process_sentence(line))

# Печатаем статистику
print("\nStatistics:")
print(f"Total tokens processed: {total_tokens}")
print(f"Tokens not found in dictionary: {not_found_tokens} ({not_found_tokens/total_tokens:.2%})")

Input:
Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.
Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.


Output:
Стала{стать=V} стабильнее{стабильнее=A} экономическая{экономический=A} и{и=CONJ|и=INTJ|и=PART|и=S} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR|иза=S} тени{тенить=V|тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=A|все=PART} Гришины{гришин=A|гришины=S} одноклассники{одноклассник=S} уже{уж=S|уже=A|уже=ADV|уже=PART} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NI} был{быть=V} чуть{чуть=ADV|чуть=CONJ} ли{ли=CONJ|ли=PART|ли=S} не{не=PART} единственным{единственный=A} кого{кто=NI} не{не=PART} вывозили{вывозить=V} никуда{никуда=ADV} дальше{дальше=A} Красной{красная=S|красный=A} Пахры{пахра=S}

Statistics:
Total tokens processed: 32
Tokens not found in di

## Решим проблему омонимов читерским способом

Для этого сначала проанализируем большой текст лемматизатором, считая его данные априорно вреными

Пример работы pymorphy3

In [ ]:
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def format_output(words_lemmas_pos):
    # words_lemmas_pos — список кортежей (слово, лемма, pos)
    result_tokens = []
    for word, lemma, pos in words_lemmas_pos:
        token_str = f"{word}{{{lemma}={pos}}}"
        result_tokens.append(token_str)
    return " ".join(result_tokens)

morph = MorphAnalyzer()

def lemmatize_and_format(sentence):
    clean_sentence = re.sub(r'[,.!?]', '', sentence)
    words = clean_sentence.split()
    formatted_tokens = []
    for w in words:
        parsed = morph.parse(w)[0]
        lemma = parsed.normal_form
        pos_full = parsed.tag.POS if parsed.tag.POS else "UNKN"
        pos_short = GRAMM_TO_SIMPLE.get(pos_full, pos_full)
        formatted_tokens.append(f"{w}{{{lemma}={pos_short}}}")
    return " ".join(formatted_tokens)

text = """Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.
Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.
"""

print(lemmatize_and_format(text))

Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S} Все{всё=PART} Гришины{гришин=S} одноклассники{одноклассник=S} уже{уже=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NI} был{быть=V} чуть{чуть=ADV} ли{ли=CONJ} не{не=PART} единственным{единственный=A} кого{кто=NI} не{не=PART} вывозили{вывозить=V} никуда{никуда=ADV} дальше{далёкий=A} Красной{красный=A} Пахры{пахра=S}


In [37]:
import re
from collections import defaultdict
import pickle
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

# версия только с частями речи
def lemmatize_sentence(sentence):
    clean_sentence = re.sub(r'[,.!?]', '', sentence)
    words = clean_sentence.split()
    pos_sequence = []
    for w in words:
        parsed = morph.parse(w)[0]
        pos_full = parsed.tag.POS
        if pos_full is None:  # пропуск, если POS нет
            continue
        pos_str = str(pos_full)
        pos_short = GRAMM_TO_SIMPLE.get(pos_str, pos_str)
        pos_sequence.append(pos_short)
    return pos_sequence

def count_pos_trigrams(pos_sequence):
    trigram_counts = defaultdict(int)
    for i in range(len(pos_sequence) - 2):
        trigram = (pos_sequence[i], pos_sequence[i+1], pos_sequence[i+2])
        trigram_counts[trigram] += 1
    return trigram_counts

INPUT_FILE = "сrime_and_punishment.txt"
all_pos = []

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            pos_seq = lemmatize_sentence(line)
            all_pos.extend(pos_seq)

trigram_dict_pos = count_pos_trigrams(all_pos)

with open("trigram_pos_dict.pkl", "wb") as f_out:
    pickle.dump(trigram_dict_pos, f_out)

print(f"Всего уникальных триграмм по POS: {len(trigram_dict_pos)}")

top10 = sorted(trigram_dict_pos.items(), key=lambda x: x[1], reverse=True)[:10]
for trigram, count in top10:
    print(trigram, count)

Всего уникальных триграмм по POS: 1840
('PR', 'A', 'S') 3048
('S', 'PR', 'S') 2391
('V', 'PR', 'S') 1928
('A', 'S', 'CONJ') 1826
('PR', 'S', 'S') 1657
('S', 'S', 'S') 1628
('A', 'S', 'S') 1619
('S', 'A', 'S') 1563
('PR', 'S', 'CONJ') 1534
('A', 'S', 'V') 1518


Далее огрномая функция, отвечающая за частотную логику

In [76]:
total_tokens = 0
not_found_tokens = 0

def get_max_trigram_freq(trigram_dict_pos, trigram_template, variant_lists):
    """
    trigram_template - троица pos с None на месте нашего слова
    variant_lists - список списков вариантов замены для позиций, где None нет
    Пример: (None, pos_j, pos_k), variant_lists = [variants_j, variants_k]
    Возвращает максимальную частоту из всех подстановок
    """
    max_freq = 0
    from itertools import product
    for combo in product(*variant_lists):
        trigram = []
        v_index = 0
        for pos in trigram_template:
            if pos is None:
                trigram.append(combo[v_index])
                v_index += 1
            else:
                trigram.append(pos)
        freq = trigram_dict_pos.get(tuple(trigram), 0)
        if freq > max_freq:
            max_freq = freq
    return max_freq


def lemmatize_disambiguate(sentence: str, ln: int, trigram_dict_pos, lemma_dict):
    import re
    global total_tokens, not_found_tokens

    clean = re.sub(r"[^\w\s]", "", sentence, flags=re.UNICODE)
    tokens = clean.split()
    length = len(tokens)

    result_tokens = []

    for i, token in enumerate(tokens):
        total_tokens += 1
        norm = normalize(token)
        variants = lemma_dict.get(norm, [(norm, "??")])
        if variants == [(norm, "??")]:
            not_found_tokens += 1

        if len(variants) == 1:
            lemma, pos = variants[0]
        else:
            best_variant = None
            best_freq = -1

            # Формируем для всех вариантов нашего слова
            for lemma_i, pos_i in variants:
                pos_i_short = GRAMM_TO_SIMPLE.get(pos_i, pos_i)
                freq_sum = 0
                count = 0

                if ln == 1:
                    # Рассматриваем триграммы с нашим словом в начале: (pos_i, pos_j, pos_k)
                    if i + 2 < length:
                        next1_norm = normalize(tokens[i + 1])
                        next2_norm = normalize(tokens[i + 2])
                        next1_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(next1_norm, [(next1_norm, "??")])]
                        next2_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(next2_norm, [(next2_norm, "??")])]

                        max_freq = get_max_trigram_freq(trigram_dict_pos, (pos_i_short, None, None), [next1_variants, next2_variants])
                        freq_sum += max_freq
                        count += 1

                elif ln == 2:
                    # Триграммы с нашим словом в конце (pos_j, pos_k, pos_i)
                    if i - 2 >= 0:
                        prev2_norm = normalize(tokens[i - 2])
                        prev1_norm = normalize(tokens[i - 1])
                        prev2_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(prev2_norm, [(prev2_norm, "??")])]
                        prev1_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(prev1_norm, [(prev1_norm, "??")])]

                        max_freq = get_max_trigram_freq(trigram_dict_pos, (None, None, pos_i_short), [prev2_variants, prev1_variants])
                        freq_sum += max_freq
                        count += 1

                    # Триграммы с нашим словом в середине (pos_j, pos_i, pos_k)
                    if i - 1 >= 0 and i + 1 < length:
                        prev1_norm = normalize(tokens[i - 1])
                        next1_norm = normalize(tokens[i + 1])
                        prev1_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(prev1_norm, [(prev1_norm, "??")])]
                        next1_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(next1_norm, [(next1_norm, "??")])]

                        max_freq = get_max_trigram_freq(trigram_dict_pos, (None, pos_i_short, None), [prev1_variants, next1_variants])
                        freq_sum += max_freq
                        count += 1

                    # Триграммы с нашим словом в начале (pos_i, pos_j, pos_k)
                    if i + 2 < length:
                        next1_norm = normalize(tokens[i + 1])
                        next2_norm = normalize(tokens[i + 2])
                        next1_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(next1_norm, [(next1_norm, "??")])]
                        next2_variants = [GRAMM_TO_SIMPLE.get(pos, pos) for _, pos in lemma_dict.get(next2_norm, [(next2_norm, "??")])]

                        max_freq = get_max_trigram_freq(trigram_dict_pos, (pos_i_short, None, None), [next1_variants, next2_variants])
                        freq_sum += max_freq
                        count += 1

                avg_freq = freq_sum / max(count, 1)

                if avg_freq > best_freq:
                    best_freq = avg_freq
                    best_variant = (lemma_i, pos_i)

            lemma, pos = best_variant

        result_tokens.append(f"{token}{{{lemma}={pos}}}")

    return " ".join(result_tokens)


In [68]:
with open("lemma_dict.pkl", "rb") as f:
    lemma_dict = pickle.load(f)
with open("trigram_pos_dict.pkl", "rb") as f:
    trigram_dict_pos = pickle.load(f)

In [78]:
sentence = "я уж не знаю, что тебе сказать"
sentence2 = "мой уж не съел лягушку"

output = lemmatize_disambiguate(sentence, ln=1, trigram_dict_pos=trigram_dict_pos, lemma_dict=lemma_dict)
print(output)
output = lemmatize_disambiguate(sentence2, ln=2, trigram_dict_pos=trigram_dict_pos, lemma_dict=lemma_dict)
print(output)

я{я=S} уж{уж=ADV} не{не=PART} знаю{знать=V} что{что=CONJ} тебе{ты=NI} сказать{сказать=V}
мой{мой=A} уж{уж=S} не{не=PART} съел{съесть=V} лягушку{лягушка=S}


## Обработка большого текста

In [79]:
INPUT_FILE = "сrime_and_punishment.txt"   # путь к файлу с текстом
OUTPUT_FILE = "сrime_and_punishment_out.txt" # куда сохранить результат

In [80]:
# Кодировки:
# Война и мир UTF-8
# Преступление и наказание windows 1251

# Сброс глобальных счётчиков перед обработкой на всякий случай
total_tokens = 0
not_found_tokens = 0

with open(INPUT_FILE, "r", encoding="UTF-8") as f:
    lines = f.readlines()

processed_lines = []
for line in lines:
    line = line.strip()
    if line:  # skip empty lines
        processed_lines.append(lemmatize_disambiguate(line, ln=1, trigram_dict_pos=trigram_dict_pos, lemma_dict=lemma_dict))

In [81]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for line in processed_lines:
        f.write(line + "\n")

print("Processed text:")
for line in processed_lines[2000:2010]:
    print(line)

# Печатаем статистику
print("\nStatistics:")
print(f"Total tokens processed: {total_tokens}")
print(f"Tokens not found in dictionary: {not_found_tokens} ({not_found_tokens/total_tokens:.2%})")

Processed text:
Она{она=NI} поминки{поминки=S} устраивает{устраивать=V}
Дас{дас=??} закуску{закуска=S} она{она=NI} вас{вы=NI} очень{очень=ADV} велела{велеть=V} благодарить{благодарить=V} что{что=CONJ} вы{вы=NI} вчера{вчера=ADV} помогли{помочь=V} нам{мы=NI} без{без=PR} вас{вы=NI} совсем{совсем=ADV} бы{бы=PART} нечем{нечего=NI} похоронить{похоронить=V} И{и=S} губы{губа=S} и{и=S} подбородок{подбородок=S} ее{она=NI} вдруг{вдруг=ADV} запрыгали{запрыгать=V} но{но=CONJ} она{она=NI} скрепилась{скрепиться=V} и{и=S} удержалась{удержаться=V} поскорей{скорее=A} опять{опять=ADV} опустив{опустить=GRND} глаза{глаз=S} в{в=PR} землю{земля=S}
Между{между=PR} разговором{разговор=S} Раскольников{раскольник=S} пристально{пристально=ADV} ее{она=NI} разглядывал{разглядывать=V} Это{это=NI} было{быть=V} худенькое{худенький=A} совсем{совсем=ADV} худенькое{худенький=A} и{и=S} бледное{бледный=A} личико{личико=S} довольно{доволен=A} неправильное{неправильный=A} какоето{какоето=??} востренькое{востренький=A} с{с=PR